### Dependencies

In [269]:
# utils 
import torch

# data
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field, LabelField, TabularDataset, BucketIterator

# model
import torch.nn as nn
import torch.nn.functional as F

# training and evaluation
import wandb
import pytorch_lightning as pl
from pytorch_lightning.metrics import Accuracy, Fbeta, Precision, Recall
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [339]:
import torchtext

In [340]:
torchtext.__version__

'0.6.0'

In [341]:
print(torch.__version__, pl.__version__)

1.5.1 1.0.3


### Models

### 1. GRU

In [253]:
class GRUClassfier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, padding_idx, hidden_size=768, num_layers=1, dropout=0.10, num_classes=3):
        super(GRUClassfier, self).__init__()
        
        # embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=padding_idx)
        # gru module
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
#             dropout=dropout,
            bidirectional=True
        )
        
        # full connected layer as classifier
        self.fc = nn.Sequential(*[
            nn.Linear(in_features=2*num_layers*hidden_size, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes)
        ])
        
    def forward(self, x, hidden=None):
        
        batch_size = x.shape[0]
        
        # get the embedding
        embedded = self.embedding(x)
        
        # pass the embedding and initial hidden states to GRU (cell state will be same as hidden states) 
        _, outputs = self.gru(embedded, hidden)
        
        # outputs.shape -> [2*num_layers, batch_size, hidden_size] convert it into batch_first format
        outputs = outputs.permute(1, 0, 2)
        outputs = outputs.reshape(batch_size, -1)
#         print(outputs.shape)
        
        # last hidden states of the BidirectionalGRU will be passed to classifier will returns logits 
        logits = self.fc(outputs)
        return logits

#### Dataset preparation for GRU

In [310]:
# create fields
TEXT = Field(lower=True, tokenize="spacy", batch_first=True)
LABEL = LabelField(batch_first=True)

In [311]:
fields = [("id", None), ("Tweets", TEXT), ("Label", LABEL)]

In [312]:
# load the train data
train = TabularDataset(
    path="../dataset/train.csv",
    format="CSV",
    fields=fields,
    skip_header=True,

)
# load the test data
test = TabularDataset(
    path="../dataset/test.csv",
    format="CSV",
    fields=fields,
    skip_header=True,

)
# take some data from test for validation during training
test, val = test.split(split_ratio=0.8)

In [313]:
# build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [314]:
# print(len(text.vocab), len(target.vocab))

In [315]:
## create data loaders
train_loader, val_loader, test_loader = BucketIterator.splits(
    datasets=(train, val, test),
    batch_size=64,
    device=device,
    shuffle=True,
    sort=False
)

In [316]:
for batch in train_loader:
    print(batch.Tweets.shape, batch.Label.shape)
    break

torch.Size([64, 35]) torch.Size([64])


In [317]:
gru = GRUClassfier(
    vocab_size=len(TEXT.vocab),
    embedding_dim=300,
    padding_idx=TEXT.vocab[TEXT.pad_token],
)

In [264]:
logits = gru(batch.Tweets)
print(logits.shape)

torch.Size([64, 1536])
torch.Size([64, 3])


In [259]:
# nn.GRU?

## Training 

In [333]:
class LightningModel(pl.LightningModule):
    
    def __init__(self, model):
        super(LightningModel, self).__init__()
        
        self.model = model

    def forward(self, x):
        logits  = self.model(x)
        return logits
    
    def configure_optimizers(self):
        return torch.optim.Adam(params=self.parameters(), lr=1e-1)
    
    def train_dataloader(self):
        return train_loader
    
    def training_step(self, batch, batch_idx):
        x, y = batch.Tweets, batch.Label
        y_ = self(x)
        loss = F.cross_entropy(y_, y)
        acc = accuracy_score(y, y_.argmax(dim=1))
        f1 = f1_score(y, y_.argmax(dim=1), average="macro")
        wandb.log({"loss":loss, "accuraccy":acc, "f1_score":f1})
        return {"loss":loss, "accuraccy":acc, "f1_score":f1}
    
    def val_dataloader(self):
        return val_loader
    
    def validation_step(self, batch, batch_idx):
        x, y = batch.Tweets, batch.Label
        y_ = self(x)
        loss = F.cross_entropy(y_, y)
        acc = accuracy_score(y, y_.argmax(dim=1))
        f1 = f1_score(y, y_.argmax(dim=1), average="macro")
        return {"val_loss":loss, "val_accuracy":torch.tensor([acc]), "val_f1":torch.tensor([f1])}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_accuracy'] for x in outputs]).mean()
        avg_f1 = torch.stack([x['val_f1'] for x in outputs]).mean()
        wandb.log({"val_loss":avg_loss, "val_accuracy":avg_acc, "val_f1":avg_f1})
        return {"val_loss":avg_loss, "val_accuracy":avg_acc, "val_f1":avg_f1}
    
    def test_dataloader(self):
        return test_loader
    
    def test_step(self, batch, batch_idx):
        x, y = batch.Tweets, batch.Label
        y_ = self(x)
        loss = F.cross_entropy(y_, y)
        acc = accuracy_score(y, y_.argmax(dim=1))
        f1 = f1_score(y, y_.argmax(dim=1), average="macro")
        precision = precision_score(y, y_.argmax(dim=1), average="macro")
        recall = recall_score(y, y_.argmax(dim=1), average="macro")
        return {"test_loss":loss, "test_precision":precision, "test_recall":recall, "test_acc":acc, "test_f1":f1}

In [334]:
model = LightningModel(model=gru)

In [335]:
logger = WandbLogger(
    name="gru",
    save_dir="./models/",
    project="hate-speech-detection",
    log_model=True,
)
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.005,
)
checkpoints = ModelCheckpoint(
    filepath="./models/{epoch}-{val_accuracy}",
    monitor="val_accuracy",
    save_top_k=1
)

In [336]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoints,
    default_root_dir="./models/",
    max_epochs=1
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [337]:
trainer.fit(model)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name  | Type         | Params
---------------------------------------
0 | model | GRUClassfier | 12 M  


Validation sanity check: 0it [00:00, ?it/s]torch.Size([64, 1536])
Epoch 0:   0%|          | 0/212 [00:00<?, ?it/s]                      torch.Size([64, 1536])


/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


Epoch 0:  95%|█████████▍| 201/212 [01:29<00:04,  2.25it/s, loss=0.851, v_num=7hkt]
Validating: 0it [00:00, ?it/s]torch.Size([64, 1536])

Epoch 0:  95%|█████████▌| 202/212 [01:29<00:04,  2.25it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])

Epoch 0:  96%|█████████▌| 203/212 [01:29<00:03,  2.26it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])

Epoch 0:  96%|█████████▌| 204/212 [01:29<00:03,  2.27it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])

Epoch 0:  97%|█████████▋| 205/212 [01:30<00:03,  2.28it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])
torch.Size([1, 1536])

Epoch 0:  98%|█████████▊| 207/212 [01:30<00:02,  2.29it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])

Validating:  64%|██████▎   | 7/11 [00:01<00:00,  6.27it/s]torch.Size([64, 1536])

Epoch 0:  99%|█████████▊| 209/212 [01:30<00:01,  2.31it/s, loss=0.851, v_num=7hkt]torch.Size([64, 1536])

Validating:  82%|████████▏ | 9/11 [00:01<00:00,  7.11it/s]torch.Size([64, 1536])

Epoch 0: 100%|█████████▉| 211/212 [01:30<

1

In [338]:
trainer.test()

Testing: 0it [00:00, ?it/s]torch.Size([64, 1536])
Testing:   5%|▍         | 2/41 [00:00<00:05,  6.57it/s]

/home/macab/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


torch.Size([64, 1536])
Testing:  46%|████▋     | 19/41 [00:02<00:02,  8.18it/s]torch.Size([5, 1536])
torch.Size([64, 1536])
Testing:  54%|█████▎    | 22/41 [00:02<00:02,  8.27it/s]torch.Size([64, 1536])
torch.Size([64, 1536])
Testing:  66%|██████▌   | 27/41 [00:03<00:01,  8.73it/s]torch.Size([64, 1536])
torch.Size([64, 1536])
Testing:  76%|███████▌  | 31/41 [00:03<00:01,  8.90it/s]torch.Size([64, 1536])
torch.Size([64, 1536])
Testing:  80%|████████  | 33/41 [00:04<00:00,  9.56it/s]torch.Size([64, 1536])
torch.Size([64, 1536])
Testing:  90%|█████████ | 37/41 [00:04<00:00,  9.27it/s]torch.Size([64, 1536])
torch.Size([64, 1536])
Testing:  98%|█████████▊| 40/41 [00:04<00:00,  8.93it/s]torch.Size([64, 1536])
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.65625,
 'test_f1': 0.2641509433962264,
 'test_loss': tensor(0.8729),
 'test_precision': 0.21875,
 'test_recall': 0.3333333333333333}
--------------------------------

[{'test_loss': 0.872868537902832,
  'test_precision': 0.21875,
  'test_recall': 0.3333333333333333,
  'test_acc': 0.65625,
  'test_f1': 0.2641509433962264},
 {'test_loss': 0.8728684186935425,
  'test_precision': 0.21875,
  'test_recall': 0.3333333333333333,
  'test_acc': 0.65625,
  'test_f1': 0.2641509433962264},
 {'test_loss': 0.8453578948974609,
  'test_precision': 0.22916666666666666,
  'test_recall': 0.3333333333333333,
  'test_acc': 0.6875,
  'test_f1': 0.2716049382716049},
 {'test_loss': 1.0694180727005005,
  'test_precision': 0.17708333333333334,
  'test_recall': 0.3333333333333333,
  'test_acc': 0.53125,
  'test_f1': 0.23129251700680273},
 {'test_loss': 0.7790889739990234,
  'test_precision': 0.25396825396825395,
  'test_recall': 0.32653061224489793,
  'test_acc': 0.75,
  'test_f1': 0.28571428571428575},
 {'test_loss': 0.8642175197601318,
  'test_precision': 0.21875,
  'test_recall': 0.3333333333333333,
  'test_acc': 0.65625,
  'test_f1': 0.2641509433962264},
 {'test_loss': 0.8